In [1]:
import rail
from rail.estimation.algos.deepdisc import *
from rail.core.data import TableHandle, JsonHandle, ModelHandle
from rail.core.stage import RailStage

/home/drewoldag/.conda/envs/rail_deepdisc39/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
DS = RailStage.data_store
DS.__class__.allow_overwrite = True

In [3]:
model_file = "./test_informer.pkl"
model_handle = DS.add_data("model", data=None, handle_class=ModelHandle, path=model_file)
test_file_for_chunks = "/home/shared/hsc/DC2/test_data/dataset_3/flattened_images_test_small.hdf5"
test_handle_for_chunks = DS.add_data("testing", data=None, handle_class=TableHandle, path=test_file_for_chunks)
metadatafile_with_chunks = "/home/shared/hsc/DC2/test_data/dataset_3/test_metadata_example.hdf5"
metadatahandle_with_chunks = DS.add_data("metadata", data=None, handle_class=Hdf5Handle, path=metadatafile_with_chunks)

cfgfile = "./configs/solo/solo_swin_DC2.py"

In [4]:
deep_estimation_dict = dict(
    chunk_size=20,
    output_dir="./",
    cfgfile = cfgfile,
    zmin=0,
    zmax=5,
    nzbins=200,
    output_mode='default',
)

EstimatorWithChunks = DeepDiscPDFEstimatorWithChunking.make_stage(
    name="DeepDiscEstimatorWithChunks",
    model=model_file,
    **deep_estimation_dict,
)

In [5]:
results_from_chunks = EstimatorWithChunks.estimate(test_handle_for_chunks, metadatahandle_with_chunks)

Caching data
Processing chunk (start:end) - (0:10)


/home/drewoldag/.conda/envs/rail_deepdisc39/lib/python3.9/site-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /data/miniconda3/envs/opence-1.7/conda-bld/pytorch-base_1672876060819/work/aten/src/ATen/native/TensorShape.cpp:3190.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


Matching objects


RuntimeError: cuDNN error: CUDNN_STATUS_INTERNAL_ERROR